# 순차 데이터 (sequential data)

<br>

- 텍스트나 시계열 데이터(time series data)와 같이 "순서"에 의미가 있는 데이터 
    - 예) "I am a boy"는 의미를 이해할 수 있지만, "boy am I a"는 의미를 이해할 수 없음. 
    - 일별 온도 데이터에서 날짜 순서가 섞인다면 내일의 온도를 쉽게 예상하기 어려움. 
    
- 감성 분석에서는 이전에 입력된 데이터를 기억하는 기능이 중요할 수 있음. 
    - 예) "별로지만 추천해요"에서 "추천해요"를 해석할 때 앞의 "별로지만"을 기억하고 있어야 무조건 긍정이라고 판단하는 실수를 하지 않음. 
    - CNN은 이런 기억 장치가 없음 (각 공간에 대한 특성이 서로 상관성, 시간성이 없음 - 독립동일분포)
        - 하나의 배치를 사용하여 정방향 연산을 수행하고나면 해당 샘플은 다음 샘플을 처리할 때 사용하지 않음.
        - 이처럼 입력 데이터의 흐름이 앞으로만 전달되는 신경망을 "피드포워드 신경망(feedforward neural network: FFNN)"이라고 함. 
        
    - 신경망이 이전에 처리했던 샘플을 다음 샘플을 처리할 떄 재사용하기 위해서는 데이터가 신경망 층에서 순환을 해야함.

# 순환 신경망 (Recurrent Neural Network; RNN)

- Elman RNN (엘만 RNN) : 가장 기본적인 RNN (1990)

<img src = "rnn.png">

## 순환 신경망 구조 

- 피드 포워드 신경망에 이전 데이터의 처리 흐름을 순환하는 고리 하나만 추가된 형태 
    - 바로 이전에 사용했던 데이터를 재사용하는 구조 

<img src = "rnn_image1.png">

                                    * 오른쪽 펼친 표현은 "BPTT(BackPropagation Through Time)"이라고도 함.

1. 첫 번째 샘플 x0를 처리하고 출력 h0가 다시 뉴런으로 입력됨. 
    - h0에는 x0에 대한 정보가 포함되어 있음. 
    
2. x1을 처리할 때 h0를 함께 사용 
    - h0과 x1을 사용해서 만든 h1에는 x0에 대한 정보가 어느정도 포함되어 있음. 
    
3. x2를 처리할 때 h1을 함께 사용 
    - 결과물 h2에는 x1과 x0에 대한 정보도 담겨있음. 
    
- 위의 과정처럼 샘플을 처리하는 각 단계를 "타임스텝(timestep)"이라고 말함.
    - 순환 신경망은 이전 타임스텝의 샘플을 기억한다. 
    
- 타임스텝이 오래될수록 순환되는 정보는 희미해짐. 

- 순환신경망에서는 layer를 "셀(cell)"이라고 부름. 
    - 하지만 입력에 가중치를 곱하고 활성화 함수를 통과시켜 다음 층으로 전달하는 기본 구조는 똑같음. 
    - 한 셀에는 여러 개의 뉴런(유닛)이 있지만, 일반적으로 모든 뉴런을 표시하지 않고 하나의 뉴런(유닛)으로 층을 표현 
    
- 셀의 출력을 "은닉 상태(hidden state)"라고 부름. 

- 입력과 출력의 길이를 다르게 설정 가능 --> 다양한 용도 
    
    1) 일대다 - 이미지 캡셔닝
    
    2) 다대일 - 스팸메일분류, 감성분석
    
    3) 다대다 - 챗봇, 번역, NER, POS_TAG

# 하이퍼볼릭 탄젠트 함수 

<img src = "tanh.png">

- 일반적으로 은닉층의 활성화 함수로는 "하이퍼볼릭 탄젠트(hyperbolic tangent)" 함수인 "tanh"가 많이 활용됨.
    - tanh 함수는 -1 ~ 1 사이의 범위를 가짐. 
    - sigmoid(0-1)와의 절충안 (시그모이드 함수의 선형 변환)
        - 입력 범위 대부분에서 아주 빠르게 포화됨. 
        - --> 그래디언트 0 되거나(그래디언트 소실 문제), 발산하여 부동소수 오버플로가 되는 문제(그래디언트 폭주 문제)
        - 이 때문에 시그모이드 활성화 함수는 거의 출력층에서만 사용 (출력을 확률로 압축)

# 가중치 공유 

<img src = "rnn_weight.png">

- 순환 신경망도 피드포워드 신경망과 같이 입력값과 가중치(U)를 곱하지만 이전 타임스텝의 은닉상태에 곱해지는 가중치(W)가 하나 더 있음. 
- 모든 타임스텝에서 사용되는 가중치는 W 하나임. 
    - 가중치 W는 타임스텝에 따라 변화되는 뉴런의 출력을 학습
    - 입력-은닉 가중치 U와 은닉-은닉 가중치 W가 연속된 타임스텝에 걸쳐 공유된다.
    - RNN이 학습하기 위해 훈련 과정에서 이런 가중치가 수정됨. 
- 맨 처음 타임스텝 1에서 사용되는 이전 은닉상태 h0은 이전 타임스텝이 없기 때문에 0으로 초기화 

- h_t = tanh( U * x_t + W * h_t-1 )
    - U : 입력-은닉 가중치
    - W : 은닉-은닉 가중치 
    - \* : 점곱
- y_t_hat = softmax( V * h_t )
    - V : 은닉-출력 가중치 

# 파라미터 수 

<img src = "rnn_param.png">

- 각 뉴런의 은닉 상태가 다음 스텝에 재사용될 때 모든 뉴런에 값이 전달됨. 
    - 즉, 이전 타임스텝의 은닉상태는 다음 타임스텝의 모든 뉴런에 완전연결됨. 
- 순환신경망의 파라미터 수 = (유닛 수 * 유닛 수) + (입력값 수 * 유닛 수) + 유닛 수

# 입력 

- 합성곱 층의 입력은 하나의 샘플이 3개의 차원을 가짐. 
    - 너비, 높이, 채널 
    
- 순환층은 일반적으로 샘플마다 2개의 차원을 가짐. 
    - 하나의 샘플을 하나의 시퀀스(sequence)라고 말함. 
    - 하나의 시퀀스 안에는 여러 개의 아이템(단어, token)이 들어 있음. 
    - 시퀀스의 길이가 타임 스텝의 길이가 됨. 
        - 예) 샘플 "I am a boy"는 4개의 아이템으로 구성된 시퀀스
        - 각 아이템을 3개의 숫자로 표현한다면 타임스텝의 크기는 (1, 4, 3)
        
- 입력이 순환층을 통과하면 순환층의 뉴런 개수만큼 출력됨.
    - 하나의 샘플은 (단어 개수, 단어 표현)의 2차원 배열 
    - 순환층을 통과하면 1차원 배열로 바뀜. 
        - 이 1차원 배열의 크기는 순환층의 뉴런의 개수 
    - 기본적으로 순환층은 마지막 타임스텝의 은닉상태만 출력으로 내보냄. 
        - 입력된 시퀀스를 모두 읽어서 정보를 마지막 은닉상태에 압축하여 전달 
        
- 만약 순환층을 여러 개 쌓는다면 
    - 순환층의 입력은 샘플마다 타임스텝과 단어 표현으로 이뤄진 2차원 배열이어야 함. 
        - 첫 번째 셀이 마지막 타임스텝의 은닉 상태만 출력한다면 입력 형식이 맞지 않음. 
        - 이 경우에는 마지막 셀을 제외한 모든 셀은 모든 타임스텝의 은닉상태를 출력 (return_sequences=True)
        - 마지막 셀은 마지막 타임 스텝의 은닉상태만 출력 

# 출력 

- 순환 신경망도 합성곱 신경망과 마찬가지로 마지막에 밀집층을 두어 클래스를 분류 
    - 다중 분류 : 출력층에 클래스 개수만큼 뉴런을 두고 소프트맥스 출력층 함수 사용 
    - 이진 분류 : 하나의 뉴런을 두고 시그모이드 출력층 함수 사용 
    
<br>

- 합성곱 신경망과의 차이점은 마지막 셀의 출력이 1차원이기 때문에 Flatten이 필요 없음. 
    - 여러 특성맵 결과를 하나로 쭉 풀어줘야 함 